In [1]:
%%time
#Standard Header used on the projects

#first the major packages used for math and graphing
import numpy as np
import matplotlib.pyplot as plt
from cycler import cycler
import scipy.special as sp

#Custome graph format style sheet
#plt.style.use('Prospectus.mplstyle')

#If being run by a seperate file, use the seperate file's graph format and saving paramaeters
#otherwise set what is needed
if not 'Saving' in locals():
    Saving = False
if not 'Titles' in locals():
    Titles = True
if not 'Ledgends' in locals():
    Ledgends = True
if not 'FFormat' in locals():
    FFormat = '.png'

#Standard cycle to make black and white images and dashed and line styles
default_cycler = (cycler('color', ['0.00', '0.40', '0.60', '0.70']) + cycler(linestyle=['-', '-', '-', '-']))
plt.rc('axes', prop_cycle=default_cycler)
my_cmap = plt.get_cmap('gray')

#Extra Headers:
import os as os
import pywt as py
import statistics as st
import os as os
import random
import multiprocessing
from joblib import Parallel, delayed
import platform
import random


from time import time as ti

import CoreFunctions as cf
#from skimage.restoration import denoise_wavelet

import os
import pickle

# currently running pid 4010813

HostName = platform.node()

if HostName == "Server":
    Computer = "Desktop"   
elif HostName[-6:] == 'wm.edu':
    Computer = "SciClone"
elif HostName == "SchoolLaptop":
    Computer = "LinLap"
elif HostName == "WTC-TAB-512":
    Computer = "PortLap"
else:
    Computer = "WinLap"

if Computer == "SciClone":
    location = '/sciclone/home20/dchendrickson01/image/'
elif Computer == "WinLap":
    location = 'C:\\Data\\'
elif Computer == "Desktop":
    location = "E:\\Backups\\Dan\\CraneData\\"
elif Computer == "LinLap":
    location = '/home/dan/Output/'
elif Computer == 'PortLap':
    location = 'C:\\users\\dhendrickson\\Desktop\\AccelData\\'

if Computer ==  "SciClone":
    rootfolder = '/sciclone/home20/dchendrickson01/'
    folder = '/scratch/RecordingsSplit/xFold/'
    #folder = '/sciclone/scr10/dchendrickson01/RecordingsSplit/xFold/'
elif Computer == "Desktop":
    rootfolder = location
    folder = rootfolder + "Recordings2\\"
elif Computer =="WinLap":
    rootfolder = location
    folder = rootfolder + "Recordings2\\"   
elif Computer == "LinLap":
    rootfolder = '/home/dan/Data/'
    folder = rootfolder + 'Recordings2/'
elif Computer =='PortLap':
    rootfolder = location 
    folder = rootfolder + 'Recordings2\\'

def Openfile(file):
    try:
        ff = open(folder+file,'rb')
        dump = pickle.load(ff)
    
        return dump[0], dump[1]
    except:
        print("bad file ",file)

CPU times: user 4.84 s, sys: 2.78 s, total: 7.62 s
Wall time: 5.5 s


In [2]:
%%time
location = folder
Titles = True
Ledgends = True

FileBatch = 20000

TimeSteps = 750
PredictSize = 25
Features = 3

MiddleLayerSize = 100
CompressedVectorSize = 20

Batches = 2

num_cores = 30
num_gpus = 1

CPU times: user 13 μs, sys: 0 ns, total: 13 μs
Wall time: 21.5 μs


In [3]:
from sys import getsizeof

In [4]:
%%time
from sklearn.model_selection import train_test_split
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Masking, Lambda, Input
from keras.models import Sequential
import tensorflow as tf

class LSTM_Autoencoder:
  def __init__(self, optimizer='adam', loss='mse'):
    self.optimizer = optimizer
    self.loss = loss
    self.n_features = Features
    self.timesteps = TimeSteps
    
  def build_model(self):
    timesteps = self.timesteps
    n_features = self.n_features
    model = Sequential()
    
    # Padding
    #model.add(Masking(mask_value=0.0, input_shape=(timesteps, n_features)))

    # Encoder
    model.add(Input(shape=(TimeSteps, Features)))
    model.add(LSTM(timesteps, activation='relu', return_sequences=True))
    model.add(LSTM(MiddleLayerSize, activation='relu', return_sequences=True))
    model.add(LSTM(CompressedVectorSize, activation='relu'))
    model.add(RepeatVector(timesteps))
    
    # Decoder
    model.add(LSTM(timesteps, activation='relu', return_sequences=True))
    model.add(LSTM(MiddleLayerSize, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(n_features)))
    
    model.compile(optimizer=self.optimizer, loss=self.loss, metrics=['accuracy'])
    model.summary()
    self.model = model
    
  def fit(self, X, epochs=3, batch_size=32):
    #self.timesteps = np.shape(X)[0]
    #self.build_model()
    
    #input_X = np.expand_dims(X, axis=1)
    self.model.fit(X, X, epochs=epochs, batch_size=batch_size)
    
  def predict(self, X):
    #input_X = np.expand_dims(X, axis=1)
    output_X = self.model.predict(X)
    reconstruction = np.squeeze(output_X)
    return np.linalg.norm(X - reconstruction, axis=-1)
  
  def plot(self, scores, timeseries, threshold=0.95):
    sorted_scores = sorted(scores)
    threshold_score = sorted_scores[round(len(scores) * threshold)]
    
    plt.title("Reconstruction Error")
    plt.plot(scores)
    plt.plot([threshold_score]*len(scores), c='r')
    plt.show()
    
    anomalous = np.where(scores > threshold_score)
    normal = np.where(scores <= threshold_score)
    
    plt.title("Anomalies")
    plt.scatter(normal, timeseries[normal][:,-1], s=3)
    plt.scatter(anomalous, timeseries[anomalous][:,-1], s=5, c='r')
    plt.show()

2024-07-02 09:33:43.142487: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-02 09:33:43.202522: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


CPU times: user 1.72 s, sys: 1.66 s, total: 3.38 s
Wall time: 4.54 s


In [5]:
%%time
lstm_autoencoder2 = LSTM_Autoencoder(optimizer='adam', loss='mse')

from tensorflow.python.keras import backend as K
config = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=num_cores,
                        inter_op_parallelism_threads=num_cores, 
                        allow_soft_placement=True,
                        device_count = {'CPU' : num_cores,
                                        'GPU' : num_gpus}
                       )
session = tf.compat.v1.Session(config=config)
K.set_session(session)

CPU times: user 46.9 ms, sys: 432 ms, total: 479 ms
Wall time: 514 ms


2024-07-02 09:33:46.912433: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22287 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:19:00.0, compute capability: 8.0


In [6]:
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        super().__init__(workers = num_cores, use_multiprocessing=True, max_queue_size=1)
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

In [7]:
lstm_autoencoder2.build_model()

2024-07-02 09:33:46.960413: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22287 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:19:00.0, compute capability: 8.0


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 750, 750)       │     2,262,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 750, 100)       │       340,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 20)             │         9,680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 750, 20)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 750, 750)       │     2,313,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 750, 100)       │       340,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 750, 3)         │           303 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,265,783 (20.09 MB)

 Trainable params: 5,265,783 (20.09 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
%%time
files = os.listdir(folder)
print('files: ', len(files))

TotalMoves=0
Loops=int(len(files)/FileBatch)+1

files:  678618
CPU times: user 200 ms, sys: 486 ms, total: 686 ms
Wall time: 2.09 s


In [9]:
%%time
k=0
start = k * FileBatch
random.shuffle(files)

Results = Parallel(n_jobs=num_cores)(delayed(Openfile)(file) for file in files[start:start+FileBatch])

X = list()

CPU times: user 1min 38s, sys: 3min 18s, total: 4min 57s
Wall time: 5min 32s


In [10]:
%%time
for result in Results:
    try:
        for j in range(len(result[0])):
            X.append(result[0][j,:,:])
            TotalMoves+=1
    except:
        print(np.shape(result[0]),result[1])
del Results

CPU times: user 2.33 s, sys: 401 ms, total: 2.73 s
Wall time: 2.74 s


In [11]:
getsizeof(X)

55621560

In [12]:
%%time
with tf.device('/cpu:0'):
    X = tf.convert_to_tensor(X, np.float32)
#y = tf.convert_to_tensor(y, np.float32)

CPU times: user 1h 3min 49s, sys: 42.9 s, total: 1h 4min 32s
Wall time: 1h 4min 23s


In [13]:
test_gen = DataGenerator(X, X, Batches)

In [14]:
getsizeof(X)

168

In [15]:
getsizeof(test_gen)

56

In [16]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.333
session = InteractiveSession(config=config)


2024-07-02 10:43:50.113571: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22287 MB memory:  -> device: 0, name: NVIDIA A30, pci bus id: 0000:19:00.0, compute capability: 8.0


In [17]:
%%time
lstm_autoencoder2.model.fit(test_gen, epochs=4, batch_size=Batches, verbose=2)
lstm_autoencoder2.model.save("LSTM_750_self.keras")

print('Total Moves ',TotalMoves)

2024-07-02 10:44:00.132215: W external/local_tsl/tsl/framework/bfc_allocator.cc:487] Allocator (GPU_0_bfc) ran out of memory trying to allocate 51.83GiB (rounded to 55654794240)requested by op StridedSlice
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-07-02 10:44:00.132286: I external/local_tsl/tsl/framework/bfc_allocator.cc:1044] BFCAllocator dump for GPU_0_bfc
2024-07-02 10:44:00.132308: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (256): 	Total Chunks: 25, Chunks in use: 24. 6.2KiB allocated for chunks. 6.0KiB in use in bin. 128B client-requested in use in bin.
2024-07-02 10:44:00.132323: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (512): 	Total Chunks: 1, Chunks in use: 1. 768B allocated for chunks. 768B in use in bin. 320B client-requested in use in bin.
2024-07-02 10:44:00.132338: I 

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run StridedSlice: Dst tensor is not initialized. [Op:StridedSlice] name: strided_slice/

In [ ]:
tic = ti()

for j in range(15):

    random.shuffle(files)

    for k in range(Loops):
        print("Starting Loop "+str(k+1)+" of "+str(Loops+1)+" elapsted time: "+str(int((ti()-tic)/100)*100))

        start = k * FileBatch

        Results = Parallel(n_jobs=num_cores)(delayed(Openfile)(file) for file in files[start:start+FileBatch])

        X = list()
        
        for result in Results:
            try:
                for j in range(len(result[0])):
                    X.append(result[0][j,:,:])
                    TotalMoves+=1
            except:
                print(np.shape(result[0]),result[1])
        del Results



        with tf.device('/cpu:0'):
            X = tf.convert_to_tensor(X, np.float32)
            #y = tf.convert_to_tensor(y, np.float32)

        test_gen = DataGenerator(X, X, Batches)
    
            
        lstm_autoencoder2.model.fit(test_gen, epochs=4, batch_size=Batches, verbose=2)
        lstm_autoencoder2.model.save("LSTM_750_self.keras")

        print('Total Moves ',TotalMoves)

        del X

Starting Loop 1 of 35 elapsted time: 0
